In [1]:
from collections import defaultdict
from dotenv import load_dotenv
from groq import Groq
import hashlib
import json
import pandas as pd
import pickle
import requests
import os
from tqdm.auto import tqdm

In [2]:
# import the documents
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
# create index for each document
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id


In [5]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [6]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [7]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [8]:
len(hashes), len(documents)

(947, 948)

In [9]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [10]:
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [11]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [12]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


After adding ids to documents, we used a LLM to generate 5 questions based on the FAQ. Here the LLM used is a Mistral model provided by Groq. In another situation, this step can be realized by human experts to have accurate informations. In absence of human experts and because our case is simple, a LLM is used to realized the ground truth dataset. 

In [13]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [14]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [15]:
client = Groq(
    api_key=GROQ_API_KEY
)

In [16]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        messages = [
            {
                "role":"user",
                "content": prompt
            }
        ],
        model="mixtral-8x7b-32768"
    )

    json_response = response.choices[0].message.content
    return json_response

In [17]:
results = {}

In [52]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/948 [00:00<?, ?it/s]

In [53]:
with open('results.bin', 'wb') as f_out:
    pickle.dump(results, f_out)

In [54]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [39]:
results['1f6520ca']

'[\n"What are the necessary requirements to take this course?",\n"Could you please guide me on the prerequisites for this course?",\n"Where can I find the details about the prerequisites for this course?",\n"Is there any information available on the required background for this course?",\n"Can you provide the location of the prerequisites for this course?"\n]'

In [55]:
parsed_results = {}

for doc_id, json_questions in results.items():
    parsed_results[doc_id] = json.loads(json_questions)

JSONDecodeError: Invalid \escape: line 2 column 95 (char 96)

In [56]:
json_questions

'[\n"Why am I receiving an error response from the Docker daemon when attempting to use the path \'\\Program Files\\Git\\var\\lib\\postgresql\\data\'?",\n"What should be the correct mounting path to use for Docker for the postgresql data directory?",\n"Can I use a relative path as the mounting path for Docker for the postgresql data directory?",\n"How do I modify the mounting path to be \'/c:/.../ny\\_taxi\\_postgres\\_data:/var/lib/postgresql/data\' in Docker?",\n"What is the significance of the leading slash in the Docker mounting path \'/e/zoomcamp/...:/var/lib/postgresql/data\'?"\n]'

In [50]:
json_question = [
    r"Why am I unable to change permissions of directory /var/lib/postgresql/data when running the provided Docker command?"
    "What is the significance of the error message 'fixing permissions on existing directory /var/lib/postgresql/data ... initdb: error: could not change permissions of directory '/var/lib/postgresql/data': Operation not permitted?",
    "How can I solve the issue of not being able to change permissions of the directory '/var/lib/postgresql/data'?",
    "Can you explain the Docker commands to create a local volume and map it to postgres data directory?",
    "What should I do if I encounter the error message 'initdb: error: directory '/var/lib/postgresql/data' exists but is not empty?"
]

In [51]:
results[doc_id] = json.dumps(json_question)

In [44]:
doc_index = {d['id']: d for d in documents}

In [ ]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [ ]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [ ]:
df.to_csv('ground-truth-data.csv', index=False)

In [ ]:
!head ground-truth-data.csv